# Parameter Tuning and Complex Model Design

In [1]:
# Redo the original bagging model for comparison purposes

In [2]:
import pickle

# Load the data from the pickle file
with open('train_test_data.pkl', 'rb') as f:
    X_train, X_test, y_train, y_test = pickle.load(f)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier

In [4]:
# Excluding the unncessary columns from the input features for the models by dropping them now
columns_to_exclude = ['ID', 'Customer_ID', 'Month', 'SSN', 'Type_of_Loan', 'Name']  # List of columns to exclude
X_train = X_train.drop(columns=columns_to_exclude)
X_test = X_test.drop(columns=columns_to_exclude)

In [5]:
# Ensemble Methods - Bagging
bagging = BaggingClassifier()
bagging.fit(X_train, y_train)
y_pred_bagging = bagging.predict(X_test)
bagging_score = accuracy_score(y_test, y_pred_bagging)
print("Bagging Classifier Accuracy:", bagging_score)

Bagging Classifier Accuracy: 0.7900834105653383


In [6]:
# Access the default parameters
default_params = bagging.get_params()

# This will help decide what hyperparameters to use in the gridsearch and what values to try based on what was used in the original model
print("Default parameters:")
for param, value in default_params.items():
    print(param, ":", value)

Default parameters:
base_estimator : deprecated
bootstrap : True
bootstrap_features : False
estimator : None
max_features : 1.0
max_samples : 1.0
n_estimators : 10
n_jobs : None
oob_score : False
random_state : None
verbose : 0
warm_start : False


# Parameter Tuning Section

The Bagging model has been chosen as the final model because of its simplicity, low computation time, and high accuracy value. In this section, we will attempt to approve upon its accuracy by using parameter tuning, and other techniques. 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters grid
param_grid = {
    'n_estimators': [200, 500, 1000],  # Number of base estimators
    'max_samples': [0.7, 1.0, 1.3],  # Number of samples to draw from X to train each base estimator
    'max_features': [0.5, 0.7, 1.0]   # Number of features to draw from X to train each base estimator
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters:", grid_search.best_params_)

# Get the best estimator
best_bagging = grid_search.best_estimator_

# Predict with the best estimator
y_pred_best_bagging = best_bagging.predict(X_test)

# Evaluate accuracy
best_bagging_score = accuracy_score(y_test, y_pred_best_bagging)
print("Tuned Bagging Classifier Accuracy:", best_bagging_score)


In [ ]:
import joblib
joblib.dump(best_bagging, 'best_bagging_model.pkl')

# Final Model Design